# Esfira Babajanyan Service Titan Internship Task

In [12]:
import pickle
import pandas as pd

invoices = []
with (open("invoices_new.pkl", "rb")) as invoice_file:
    while True:
        try:
            invoices = pickle.load(invoice_file)
        except EOFError:
            break
            
print(str(invoices[1]))
print(len(invoices))

{'id': 389528, 'created_on': '2019-03-19', 'items': [{'item': {'id': 172535, 'name': 'ii_172535', 'unit_price': 105, 'type': 0}, 'quantity': 4}, {'item': {'id': 175208, 'name': 'ii_175208', 'unit_price': 109, 'type': 0}, 'quantity': 7}, {'item': {'id': 139970, 'name': 'ii_139970', 'unit_price': 193, 'type': 2}, 'quantity': 4}, {'item': {'id': 156273, 'name': 'ii_156273', 'unit_price': 129, 'type': 2}, 'quantity': 6}, {'item': {'id': 178512, 'name': 'ii_178512', 'unit_price': 108, 'type': 1}, 'quantity': 7}, {'item': {'id': 191045, 'name': 'ii_191045', 'unit_price': 195, 'type': 1}, 'quantity': 8}, {'item': {'id': 189841, 'name': 'ii_189841', 'unit_price': 171, 'type': 1}, 'quantity': 4}]}
100


In [217]:
class DataExtractor:
    
    def __init__(self, path_invoices, path_expired = None):
        self.path_invoices = path_invoices
        self.path_expired = path_expired
        
    def load_invoices(self):
        invoices = []
        with (open(self.path_invoices, "rb")) as invoice_file:
            while True:
                try:
                    invoices = pickle.load(invoice_file)
                except EOFError:
                    break
        return invoices
    
    def load_expired(self):
        expired_ids = []
        with open(self.path_expired, 'r') as f:
            ids = f.read()
            expired_ids = ids.split(", ")
        return expired_ids
    
    def calculate_invoice(self, invoice):
        total = 0 
        try:
            items = invoice["items"]
        except KeyError:
            print(f"'items' key not found in invoice: {invoice}")
            return
            
        for item in invoice["items"]:
            try:
                quantity = int(item["quantity"])
                unit_price = int(item["item"]["unit_price"])
            except ValueError:
                print("Invalid value")
                continue           
                
            total += quantity * unit_price
            
        return total
                    
    def transform_invoices(self, invoices):
        item_types = {0: "Material", 1: "Equipment", 2: "Service", 3: "Other"}
        expired_ids = self.load_expired()
        all_items = []
        
        for i in invoices:
            total_invoice_price = self.calculate_invoice(i)
            
            if "items" not in i:
                print(f"'items' key not found in invoice: {i}")
                continue
            
            for item in i["items"]:
                new_item = {}
                new_item["invoice_id"] = i["id"]
                new_item["created_on"] = i["created_on"]
                new_item["invoiceitem_id"] = item["item"]["id"]
                new_item["invoiceitem_name"] = item["item"]["name"]
                new_item["type"] = item_types.get(item["item"]["type"], "Other")
                new_item["unit_price"] = item["item"]["unit_price"]
                new_item["total_price"] = item["item"]["unit_price"] * item["quantity"]
                
                try:
                    total_price = float(new_item["total_price"])
                    new_item["percentage_in_invoice"] = (total_price / total_invoice_price) * 100
                except ValueError:
                    print("Invalid total price")
                    continue
                
                new_item["percentage_in_invoice"] = float(new_item["total_price"] / total_invoice_price) * 100
                new_item["is_expired"] = (str(i["id"]) in expired)
                all_items.append(new_item)
        
        return all_items

In [218]:
invoices_path = "C:\\Users\\Admin\\Downloads\\data\\invoices_new.pkl"
expired_path = "C:\\Users\\Admin\\Downloads\\data\\expired_invoices.txt"
extracted = DataExtractor(invoices_path, expired_path)


In [219]:
print(extracted)

In [220]:
invoices = extracted.load_invoices()

In [221]:
expired = extracted.load_expired()

In [223]:
transformed_invoices = extracted.transform_invoices(invoices)

Invalid value
Invalid total price
'items' key not found in invoice: {'id': 379961, 'created_on': '2019-05-12'}
'items' key not found in invoice: {'id': 379961, 'created_on': '2019-05-12'}
'items' key not found in invoice: {'id': 350506, 'created_on': '2019-01-24'}
'items' key not found in invoice: {'id': 350506, 'created_on': '2019-01-24'}
Invalid value
Invalid total price
Invalid value
Invalid total price
Invalid value
Invalid total price


In [225]:
df = pd.DataFrame(transformed_invoices)

In [226]:
df

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,percentage_in_invoice,is_expired
0,365371O,2019-05-27,142446,ii_142446,Equipment,182,1092,86.119874,False
1,365371O,2019-05-27,154019,ii_154019,Equipment,176,176,13.880126,False
2,389528,2019-03-19,172535,ii_172535,Material,105,420,7.331122,True
3,389528,2019-03-19,175208,ii_175208,Material,109,763,13.318206,True
4,389528,2019-03-19,139970,ii_139970,Service,193,772,13.475301,True
...,...,...,...,...,...,...,...,...,...
483,321243,2019-05-26,175528,ii_175528,Equipment,169,507,23.203661,False
484,377960,2019-02-22,123242,ii_123242,Equipment,110,550,16.471998,True
485,377960,2019-02-22,196386,ii_196386,Other,150,900,26.954178,True
486,377960,2019-02-22,196707,ii_196707,Service,191,573,17.160827,True


In [227]:
sorted_df = df.sort_values(by=['invoice_id', 'invoiceitem_id'], ascending=[True, True])

In [228]:
sorted_df.head()

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,percentage_in_invoice,is_expired
291,301695,2019-04-26,103215,ii_103215,Other,135,945,37.485125,False
289,301695,2019-04-26,166227,ii_166227,Equipment,118,708,28.084094,False
292,301695,2019-04-26,171394,ii_171394,Other,128,128,5.077350,False
290,301695,2019-04-26,195625,ii_195625,Other,148,740,29.353431,False
129,304245,2019-03-17,121446,ii_121446,Service,158,632,12.842918,False


In [229]:
sorted_df.to_csv('C:\\Users\\Admin\\Desktop\\ServiceTitan\\invoices_dataset.csv', index=False)

## Thank you :)